In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from datetime import datetime

from multiprocessing import cpu_count

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Subset, DataLoader

from torch.distributions import *

import skorch

%load_ext tensorboard
torch.autograd.set_detect_anomaly(True)

## Dataset

In [3]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': cpu_count(), 'pin_memory': True} if use_cuda else {}
train_loader = DataLoader(Subset(
    datasets.MNIST('/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), indices=range(10000)),
    batch_size=64, shuffle=True, **kwargs)
test_loader = DataLoader(Subset(
    datasets.MNIST('/data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), indices=range(10000)),
    batch_size=1000, shuffle=True, **kwargs)

0it [00:00, ?it/s]

9920512it [00:09, 1031265.59it/s]                             


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 50694.81it/s]                            
0it [00:00, ?it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw


1654784it [00:02, 739497.21it/s]                             
0it [00:00, ?it/s]

Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw


8192it [00:00, 21996.20it/s]            


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw
Processing...
Done!


## Generic autoencoder class

In [7]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        now = datetime.now()
        current_time = now.strftime("%Y%m%d-%H%M%S")
        self.writer = SummaryWriter(log_dir="/data/runs/"+current_time)
    
    def trains(self, device, train_loader, optimizer, epoch):
        self.train()
        loss_sum = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = self.compute_loss_train(data, target)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
            self.writer.add_scalar('Loss/train', loss.item(), epoch*len(train_loader)+batch_idx)
            
    def tests(self, device, test_loader):
        self.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                loss, output = self.compute_loss_test(data, target)
                test_loss += loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

## Autoencoder

In [8]:
class SimpleAutoencoder(Autoencoder):
    def __init__(self):
        super(SimpleAutoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output       
    
    def compute_loss_train(self, data, target):
        output = self(data)
        return F.nll_loss(output, target)
    
    def compute_loss_test(self, data, target):
        output = self(data)
        return F.nll_loss(output, target, reduction='sum').item(), output  # sum up batch loss

In [9]:
model = SimpleAutoencoder().to(device)
optimizer = optim.Adadelta(model.parameters())

# plot model
dataiter = iter(train_loader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# write to tensorboard
#writer.add_image('mnist_images', img_grid)

scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 2.291134
Train Epoch: 1 [640/10000 (6%)]	Loss: 1.352269
Train Epoch: 1 [1280/10000 (13%)]	Loss: 0.935395
Train Epoch: 1 [1920/10000 (19%)]	Loss: 0.671355
Train Epoch: 1 [2560/10000 (25%)]	Loss: 0.555808
Train Epoch: 1 [3200/10000 (32%)]	Loss: 0.276792
Train Epoch: 1 [3840/10000 (38%)]	Loss: 0.350703
Train Epoch: 1 [4480/10000 (45%)]	Loss: 0.517797
Train Epoch: 1 [5120/10000 (51%)]	Loss: 0.378069
Train Epoch: 1 [5760/10000 (57%)]	Loss: 0.604044
Train Epoch: 1 [6400/10000 (64%)]	Loss: 0.415700
Train Epoch: 1 [7040/10000 (70%)]	Loss: 0.373399
Train Epoch: 1 [7680/10000 (76%)]	Loss: 0.344828
Train Epoch: 1 [8320/10000 (83%)]	Loss: 0.210124
Train Epoch: 1 [8960/10000 (89%)]	Loss: 0.387064


KeyboardInterrupt: 

## Gaussian Variational Autoencoder

In [ ]:
class VAE(Autoencoder):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(9216, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    def loss_function(self, recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD
    
    def compute_loss_train(self, data, target):
        recon_batch, mu, logvar = self(data)
        return self.loss_function(recon_batch, data, mu, logvar)
    
    def compute_loss_test(self, data, target):
        recon_batch, mu, logvar = self(data)
        return self.loss_function(recon_batch, data, mu, logvar).item(), recon_batch  # sum up batch loss

In [ ]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

## Stick-breaking process

In [ ]:
def stickbreakingprocess(k, a, b):
    uniform_samples = Uniform(torch.tensor([0.0]), torch.tensor([1.0])).rsample(torch.tensor([a.size()[0],k])).view(-1,k)
    exp_a = 1/a
    exp_b = 1/b
    km = (1- uniform_samples.pow(exp_b)).pow(exp_a)
    
    #no Nans are allowed in the matrix
    assert not torch.isnan(km).any().item()
    
    latent_variables = km.clone()
    for i in range(1,k):
        latent_variables[:,i] = (1-km[:,i-1])*km[:,i]
    return latent_variables

In [377]:
stickbreakingprocess(20, torch.rand(10,20), torch.rand(10,20)).sum(axis=1)

tensor([4.9755, 5.9673, 5.4244, 6.5958, 5.4054, 5.2247, 5.3552, 5.8199, 4.8526,
        4.7155])

In [378]:
rn = torch.rand(128, 20)
exp = torch.stack(20*[torch.rand(128)]).T
print(exp.size())
torch.pow(rn, exp)

torch.Size([128, 20])


tensor([[0.6310, 0.7982, 0.5497,  ..., 0.9551, 0.4052, 0.6159],
        [0.7992, 0.8711, 0.9738,  ..., 0.8562, 0.9381, 0.9507],
        [0.8211, 0.7767, 0.9172,  ..., 0.5309, 0.9952, 0.8701],
        ...,
        [0.9452, 0.2655, 0.5174,  ..., 0.8715, 0.3603, 0.7503],
        [0.4043, 0.9054, 0.7716,  ..., 0.9230, 0.8147, 0.9134],
        [0.2544, 0.3529, 0.7620,  ..., 0.2478, 0.4003, 0.6405]])

## Stick-breaking Autoencoder

In [379]:
class SBVAE(Autoencoder):
    def __init__(self, k):
        super(SBVAE, self).__init__()
        self.k = k
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(9216, 400)
        self.norm = nn.BatchNorm1d(400)
        self.fc21 = nn.Linear(400, k)
        self.fc22 = nn.Linear(400, k)
        
        
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        h1 = F.relu(self.fc1(x))
        h1_norm = self.norm(h1)
        return F.softplus(self.fc21(h1_norm)), F.softplus(self.fc22(h1_norm))

    def reparameterize(self, a, b):
        return stickbreakingprocess(20, a, b)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        a, b = self.encode(x)
        z = self.reparameterize(a, b)
        return self.decode(z), a, b
    
    def loss_function(self, recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE 
    
    def compute_loss_train(self, data, target):
        recon_batch, a, b = self(data)
        return self.loss_function(recon_batch, data, a, b)
    
    def compute_loss_test(self, data, target):
        recon_batch, a, b = self(data)
        return self.loss_function(recon_batch, data, a, b).item(), recon_batch  # sum up batch loss

In [ ]:
model = SBVAE(k=20).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 34917.019531
Train Epoch: 1 [640/10000 (6%)]	Loss: 13923.922852
Train Epoch: 1 [1280/10000 (13%)]	Loss: -14046.452148
Train Epoch: 1 [1920/10000 (19%)]	Loss: -46873.195312
Train Epoch: 1 [2560/10000 (25%)]	Loss: -88523.187500
Train Epoch: 1 [3200/10000 (32%)]	Loss: -146552.718750
Train Epoch: 1 [3840/10000 (38%)]	Loss: -231216.484375
Train Epoch: 1 [4480/10000 (45%)]	Loss: -335954.406250
Train Epoch: 1 [5120/10000 (51%)]	Loss: -412340.718750
Train Epoch: 1 [5760/10000 (57%)]	Loss: -473755.375000
Train Epoch: 1 [6400/10000 (64%)]	Loss: -473197.093750
Train Epoch: 1 [7040/10000 (70%)]	Loss: -479289.406250
Train Epoch: 1 [7680/10000 (76%)]	Loss: -505992.093750
Train Epoch: 1 [8320/10000 (83%)]	Loss: -463762.000000
Train Epoch: 1 [8960/10000 (89%)]	Loss: -476709.968750
Train Epoch: 1 [9600/10000 (96%)]	Loss: -482667.562500

Test set: Average loss: -7867.4991, Accuracy: 0/10000 (0%)

Train Epoch: 2 [0/10000 (0%)]	Loss: -494819.187500
Train Epoch: 2 [640/1